In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install schedulefree optuna -q

In [ ]:
train=pd.read_csv("/kaggle/input/eds-232-ocean-chemistry-prediction-for-calcofi/train.csv")
test=pd.read_csv("/kaggle/input/eds-232-ocean-chemistry-prediction-for-calcofi/test.csv")
sub=pd.read_csv("/kaggle/input/eds-232-ocean-chemistry-prediction-for-calcofi/sample_submission.csv")

In [ ]:
import torch
import torch.nn as nn
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import numpy as np
import random

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Drop unneeded columns and handle missing values
train = train.drop(columns=["Unnamed: 12", "id"])  # Dropping unnecessary columns

# CRITICAL FIX: Rename TA1.x to TA1 to match test data
train = train.rename(columns={"TA1.x": "TA1"})

# ============================================
# Feature Engineering: Add Squared Terms
# ============================================
print("="*60)
print("FEATURE ENGINEERING")
print("="*60)

# Extract features and target
feature_columns = [col for col in train.columns if col != 'DIC']
X = train[feature_columns].copy()
y = train['DIC'].values
X_test = test[feature_columns].copy()

print(f"\nOriginal features: {X.shape}")

# Add squared terms for main features (high correlation with DIC)
main_features = ['PO4uM', 'NO3uM', 'SiO3uM', 'Temperature', 'Salinity']

print(f"\nAdding squared terms for: {main_features}")

for feat in main_features:
    if feat in X.columns:
        X[f'{feat}^2'] = X[feat] ** 2
        X_test[f'{feat}^2'] = X_test[feat] ** 2
        print(f"  Added: {feat}^2")

print(f"\nFinal feature shape: {X.shape}")
print(f"Features: {list(X.columns)}")

# Convert to numpy arrays
X = X.values
X_test = X_test.values

# Normalize features
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
X_test_scaled = scaler_X.transform(X_test)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

print(f"\nNormalized features:")
print(f"  X_scaled shape: {X_scaled.shape}")
print(f"  y_scaled shape: {y_scaled.shape}")

# 5-Fold Cross Validation setup
N_FOLDS = 5
kfold = KFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

print(f"\n5-Fold Cross Validation:")
print(f"  Number of folds: {N_FOLDS}")
print(f"  Total samples: {len(X_scaled)}")
print(f"  Approx samples per fold: {len(X_scaled) // N_FOLDS}")
print("="*60)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class OceanChemistryDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        # Handle both numpy arrays and pandas Series
        if hasattr(y, 'values'):
            y = y.values
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

print("Dataset class defined - will be used in 5-fold CV")

In [ ]:
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_size=128, dropout_rate=0.0, activation='relu'):
        super(MLPModel, self).__init__()

        # Activation function mapping
        activation_map = {
            'relu': nn.ReLU(),
            'leaky_relu': nn.LeakyReLU(0.1),
            'elu': nn.ELU(),
            'gelu': nn.GELU(),
            'silu': nn.SiLU(),  # Swish
            'tanh': nn.Tanh()
        }

        # Select activation function
        self.activation = activation_map.get(activation, nn.ReLU())

        # Determine initialization based on activation
        nonlinearity = 'relu' if activation in ['relu', 'leaky_relu'] else 'linear'

        # Single hidden layer with Batch Normalization
        self.hidden = nn.Linear(input_size, hidden_size)
        if nonlinearity == 'relu':
            nn.init.kaiming_normal_(self.hidden.weight, mode='fan_in', nonlinearity='relu')
        else:
            nn.init.xavier_normal_(self.hidden.weight)
        nn.init.constant_(self.hidden.bias, 0)
        
        # Batch Normalization
        self.bn = nn.BatchNorm1d(hidden_size)
        
        self.dropout = nn.Dropout(dropout_rate)
        
        # Output layer
        self.output = nn.Linear(hidden_size, 1)
        nn.init.xavier_normal_(self.output.weight)
        nn.init.constant_(self.output.bias, 0)
        
        # Residual connection (shortcut from input to output)
        self.shortcut = nn.Linear(input_size, 1)
        nn.init.xavier_normal_(self.shortcut.weight)
        nn.init.constant_(self.shortcut.bias, 0)

    def forward(self, x):
        # Main path: input -> hidden -> batch_norm -> activation -> dropout -> output
        h = self.hidden(x)
        h = self.bn(h)
        h = self.activation(h)
        h = self.dropout(h)
        main_output = self.output(h)
        
        # Residual path: input -> output (shortcut)
        residual = self.shortcut(x)
        
        # Add residual connection
        output = main_output + residual
        
        return output

# Set device (GPU if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

print(f"\nMLP Model Architecture:")
print(f"  Input -> Hidden -> BatchNorm -> Activation -> Dropout -> Output")
print(f"  + Residual connection (Input -> Output)")
print(f"  Dropout: Fixed to 0.0")

In [ ]:
import torch.optim as optim

# Hyperparameters
hidden_size = 128
dropout_rate = 0.0
activation = 'relu'
loss_function = 'mse'
lr = 0.01
weight_decay = 1e-4
batch_size = 64
scheduler_name = 'reduce_plateau'
epochs = 2000

print("="*60)
print("5-FOLD CROSS VALIDATION TRAINING")
print("="*60)
print(f"Hidden size: {hidden_size}")
print(f"Dropout: {dropout_rate}")
print(f"Activation: {activation}")
print(f"Loss: {loss_function}")
print(f"Learning rate: {lr}")
print(f"Weight decay: {weight_decay}")
print(f"Batch size: {batch_size}")
print(f"Scheduler: {scheduler_name}")
print(f"Epochs: {epochs}")
print("="*60)

# Loss function
loss_map = {
    'mse': nn.MSELoss(),
    'mae': nn.L1Loss(),
    'smooth_l1': nn.SmoothL1Loss(),
    'huber': nn.HuberLoss()
}
criterion = loss_map[loss_function]

# Store fold results
fold_scores = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(X_scaled)):
    print(f"\n{'='*60}")
    print(f"FOLD {fold + 1}/{N_FOLDS}")
    print(f"{'='*60}")
    
    # Split data
    X_train_fold = X_scaled[train_idx]
    y_train_fold = y_scaled[train_idx]
    X_val_fold = X_scaled[val_idx]
    y_val_fold = y_scaled[val_idx]
    
    print(f"Train samples: {len(X_train_fold)}, Val samples: {len(X_val_fold)}")
    
    # Create datasets and dataloaders
    train_dataset = OceanChemistryDataset(X_train_fold, y_train_fold)
    val_dataset = OceanChemistryDataset(X_val_fold, y_val_fold)
    
    g = torch.Generator()
    g.manual_seed(SEED)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, generator=g)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Initialize model
    model = MLPModel(input_size=X_scaled.shape[1], hidden_size=hidden_size, 
                     dropout_rate=dropout_rate, activation=activation)
    model = model.to(device)
    
    # Optimizer
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    # Scheduler
    if scheduler_name == 'cosine':
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
    elif scheduler_name == 'reduce_plateau':
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
    elif scheduler_name == 'step':
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.5)
    elif scheduler_name == 'exponential':
        scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)
    else:
        scheduler = None
    
    # Training loop
    best_val_rmse = float('inf')
    
    for epoch in range(epochs):
        # Training
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs.squeeze(), y_batch)
            loss.backward()
            optimizer.step()
        
        # Validation
        model.eval()
        val_predictions = []
        val_targets = []
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                val_predictions.extend(outputs.squeeze().cpu().numpy())
                val_targets.extend(y_batch.cpu().numpy())
        
        # Calculate RMSE in original scale
        val_predictions_original = scaler_y.inverse_transform(np.array(val_predictions).reshape(-1, 1)).flatten()
        val_targets_original = scaler_y.inverse_transform(np.array(val_targets).reshape(-1, 1)).flatten()
        val_rmse = np.sqrt(np.mean((val_predictions_original - val_targets_original)**2))
        
        # Update learning rate scheduler
        if scheduler is not None:
            if scheduler_name == 'reduce_plateau':
                scheduler.step(val_rmse)
            else:
                scheduler.step()
        
        # Track best
        if val_rmse < best_val_rmse:
            best_val_rmse = val_rmse
        
        # Print progress
        if epoch % 200 == 0:
            current_lr = optimizer.param_groups[0]['lr']
            print(f"  Epoch {epoch+1}/{epochs} | Val RMSE: {val_rmse:.4f} | Best: {best_val_rmse:.4f} | LR: {current_lr:.6f}")
    
    fold_scores.append(best_val_rmse)
    print(f"\nFold {fold + 1} Best RMSE: {best_val_rmse:.4f}")

# Summary
print(f"\n{'='*60}")
print("5-FOLD CV RESULTS")
print(f"{'='*60}")
for i, score in enumerate(fold_scores):
    print(f"Fold {i+1}: {score:.4f}")
print(f"\nMean RMSE: {np.mean(fold_scores):.4f} ± {np.std(fold_scores):.4f}")
print(f"{'='*60}")

# Train final model on all data
print(f"\nTraining final model on all data...")
model = MLPModel(input_size=X_scaled.shape[1], hidden_size=hidden_size, 
                 dropout_rate=dropout_rate, activation=activation)
model = model.to(device)

optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

if scheduler_name == 'reduce_plateau':
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
elif scheduler_name != 'none':
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
else:
    scheduler = None

full_dataset = OceanChemistryDataset(X_scaled, y_scaled)
g = torch.Generator()
g.manual_seed(SEED)
full_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=True, generator=g)

for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in full_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs.squeeze(), y_batch)
        loss.backward()
        optimizer.step()
    
    if scheduler is not None and scheduler_name != 'reduce_plateau':
        scheduler.step()

print("Final model training complete!")

In [ ]:
# ============================================
# Optuna Hyperparameter Optimization with 5-Fold CV
# ============================================

import optuna
from optuna.trial import Trial
from optuna.pruners import HyperbandPruner

def objective(trial: Trial):
    # Optimizer selection
    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'adamw', 'radam'])
    
    # Scheduler selection
    scheduler_name = trial.suggest_categorical('scheduler', 
        ['none', 'cosine', 'reduce_plateau', 'step', 'exponential'])
    
    # Hyperparameters to optimize
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-1, log=True)
    dropout_rate = 0.0  # Fixed to 0
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    
    # Adam betas (momentum coefficients)
    beta1 = trial.suggest_float('beta1', 0.8, 0.95)
    beta2 = trial.suggest_float('beta2', 0.9, 0.9999)
    
    # Scheduler-specific parameters
    if scheduler_name == 'cosine':
        T_max = trial.suggest_int('T_max', 50, 500)
    elif scheduler_name == 'reduce_plateau':
        lr_factor = trial.suggest_float('lr_factor', 0.1, 0.5)
        lr_patience = trial.suggest_int('lr_patience', 10, 50)
    elif scheduler_name == 'step':
        step_size = trial.suggest_int('step_size', 100, 500)
        gamma = trial.suggest_float('gamma', 0.1, 0.9)
    elif scheduler_name == 'exponential':
        exp_gamma = trial.suggest_float('exp_gamma', 0.95, 0.9999)
    
    # Activation function selection
    activation = trial.suggest_categorical('activation', ['relu', 'leaky_relu', 'elu', 'gelu', 'silu', 'tanh'])
    
    # Loss function selection
    loss_function = trial.suggest_categorical('loss_function', ['mse', 'mae', 'smooth_l1', 'huber'])
    
    # Single hidden layer size
    hidden_size = trial.suggest_categorical('hidden_size', [32, 64, 128, 256, 512])
    
    # Select loss function
    loss_map = {
        'mse': nn.MSELoss(),
        'mae': nn.L1Loss(),
        'smooth_l1': nn.SmoothL1Loss(),
        'huber': nn.HuberLoss()
    }
    criterion_trial = loss_map[loss_function]
    
    # 5-Fold Cross Validation
    fold_rmses = []
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X_scaled)):
        # Split data
        X_train_fold = X_scaled[train_idx]
        y_train_fold = y_scaled[train_idx]
        X_val_fold = X_scaled[val_idx]
        y_val_fold = y_scaled[val_idx]
        
        # Create datasets and dataloaders
        train_dataset_trial = OceanChemistryDataset(X_train_fold, y_train_fold)
        val_dataset_trial = OceanChemistryDataset(X_val_fold, y_val_fold)
        
        g_trial = torch.Generator()
        g_trial.manual_seed(SEED)
        
        train_loader_trial = DataLoader(train_dataset_trial, batch_size=batch_size, shuffle=True, generator=g_trial)
        val_loader_trial = DataLoader(val_dataset_trial, batch_size=batch_size, shuffle=False)
        
        # Create model
        model_trial = MLPModel(input_size=X_scaled.shape[1], hidden_size=hidden_size, 
                              dropout_rate=dropout_rate, activation=activation)
        model_trial = model_trial.to(device)
        
        # Create optimizer based on selection
        if optimizer_name == 'adam':
            optimizer_trial = optim.Adam(
                model_trial.parameters(), 
                lr=lr, 
                betas=(beta1, beta2),
                weight_decay=weight_decay
            )
        elif optimizer_name == 'adamw':
            optimizer_trial = optim.AdamW(
                model_trial.parameters(), 
                lr=lr, 
                betas=(beta1, beta2),
                weight_decay=weight_decay
            )
        elif optimizer_name == 'radam':
            optimizer_trial = optim.RAdam(
                model_trial.parameters(), 
                lr=lr, 
                betas=(beta1, beta2),
                weight_decay=weight_decay
            )
        
        # Create scheduler based on selection
        if scheduler_name == 'cosine':
            scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer_trial, T_max=T_max)
        elif scheduler_name == 'reduce_plateau':
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                optimizer_trial, mode='min', factor=lr_factor, patience=lr_patience)
        elif scheduler_name == 'step':
            scheduler = optim.lr_scheduler.StepLR(optimizer_trial, step_size=step_size, gamma=gamma)
        elif scheduler_name == 'exponential':
            scheduler = optim.lr_scheduler.ExponentialLR(optimizer_trial, gamma=exp_gamma)
        else:
            scheduler = None
        
        # Early stopping
        best_val_rmse = float('inf')
        patience = 100
        patience_counter = 0
        
        # Training loop
        max_epochs = 2000
        for epoch in range(max_epochs):
            # Training
            model_trial.train()
            for X_batch, y_batch in train_loader_trial:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                optimizer_trial.zero_grad()
                outputs = model_trial(X_batch)
                loss = criterion_trial(outputs.squeeze(), y_batch)
                loss.backward()
                optimizer_trial.step()
            
            # Validation
            model_trial.eval()
            val_predictions = []
            val_targets = []
            with torch.no_grad():
                for X_batch, y_batch in val_loader_trial:
                    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                    outputs = model_trial(X_batch)
                    val_predictions.extend(outputs.squeeze().cpu().numpy())
                    val_targets.extend(y_batch.cpu().numpy())
            
            # Calculate RMSE in original scale
            val_predictions_original = scaler_y.inverse_transform(np.array(val_predictions).reshape(-1, 1)).flatten()
            val_targets_original = scaler_y.inverse_transform(np.array(val_targets).reshape(-1, 1)).flatten()
            val_rmse = np.sqrt(np.mean((val_predictions_original - val_targets_original)**2))
            
            # Update learning rate scheduler
            if scheduler is not None:
                if scheduler_name == 'reduce_plateau':
                    scheduler.step(val_rmse)
                else:
                    scheduler.step()
            
            # Early stopping
            if val_rmse < best_val_rmse:
                best_val_rmse = val_rmse
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    break
            
            # Report intermediate value for Hyperband pruning (only for first fold)
            if fold == 0:
                trial.report(val_rmse, epoch)
                
                # Handle pruning
                if trial.should_prune():
                    raise optuna.TrialPruned()
        
        fold_rmses.append(best_val_rmse)
    
    # Return mean RMSE across all folds
    mean_rmse = np.mean(fold_rmses)
    return mean_rmse

# Create study with Hyperband Pruner
pruner = HyperbandPruner(
    min_resource=50,
    max_resource=2000,
    reduction_factor=3
)

study = optuna.create_study(
    direction='minimize', 
    study_name='mlp_5fold_cv_optimization',
    pruner=pruner
)

print("="*60)
print("Starting Optuna optimization with 5-Fold CV")
print("="*60)
print(f"Settings: min_resource=50, max_resource=2000, reduction_factor=3")
print(f"Optimizing: optimizer (Adam/AdamW/RAdam),")
print(f"            scheduler (None/Cosine/ReducePlateau/Step/Exponential),")
print(f"            lr, weight_decay, batch_size, beta1, beta2,")
print(f"            activation, loss_function, hidden_size")
print(f"Model: Single-layer MLP with Residual Connection + BatchNorm")
print(f"Input: Original features + squared terms (21 dimensions)")
print(f"Dropout: Fixed to 0.0")
print(f"Evaluation: 5-Fold Cross Validation (mean RMSE)")
print("="*60)

study.optimize(objective, n_trials=100, timeout=7200)  # 100 trials or 2 hours

# Print best parameters
print("\n" + "="*60)
print("OPTIMIZATION COMPLETE!")
print("="*60)
print(f"\nBest trial:")
print(f"  Mean CV RMSE: {study.best_trial.value:.4f}")
print(f"\n  Best Parameters:")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")

# Print pruning statistics
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
print(f"\n  Statistics:")
print(f"    Completed trials: {len(complete_trials)}")
print(f"    Pruned trials: {len(pruned_trials)}")
print(f"    Total trials: {len(study.trials)}")
print(f"    Pruning efficiency: {len(pruned_trials)/len(study.trials)*100:.1f}%")
print("="*60)

In [ ]:
# Convert the test set into a torch tensor and move to GPU
test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

# Set the model to evaluation mode
model.eval()

# Make predictions
with torch.no_grad():
    predictions_scaled = model(test_tensor).squeeze().cpu().numpy()
    # Inverse transform to get actual DIC values
    predictions = scaler_y.inverse_transform(predictions_scaled.reshape(-1, 1)).flatten()

print(f"Predictions - Min: {predictions.min():.2f}, Max: {predictions.max():.2f}, Mean: {predictions.mean():.2f}")

# Prepare submission
submission = pd.DataFrame({"id": range(1455, 1455 + len(predictions)), "DIC": predictions})
submission.to_csv("submission-26.csv", index=False)
print("Submission saved!")